In [1]:
import pandas #导入模块1 pandas 
import netCDF4 as ne  #导入模块2 netCDF4
import numpy as np    #导入模块3 numpy
import  csv #导入模块4 csv
import datetime #导入模块5 datatime
import sys #导入系统自带模块6 sys（不用下载）
import matplotlib.pyplot as plt  #（不用下载）
from mpl_toolkits.basemap import Basemap  #导入模块7 basemap
from pandas import DataFrame  #导入数据框

In [2]:
source_file='D:/citespace/prec_CMFD_V0106_B-01_03hr_010deg_197901.nc'#nc文件路径，注意需要中文路径；斜线\改成/
dataset=ne.Dataset(source_file)  #将源文件转换为文本数据集
#print(dataset.variables.keys())  #此处可以取消print前的#，查看数据集的属性。
                                  #获取相应数组集合--纬度经度温度深度
lat = dataset.variables['lat'][:] #经度读取并储存在自己定义的lat变量中
lon = dataset.variables['lon'][:] #维度读取并储存在自己定义的lon变量中
time=dataset.variables['time'] [:]   #时间读取并储存在自己定义的time变量中
prec=dataset.variables['prec'][:]  #降雨读取并储存在自己定义的prec变量中

In [ ]:
for var in dataset.variables.keys(): #对于在数据集里的每个var（属性）
    data=dataset.variables[var][:].data  #定义数据data储存数据集里每个var（属性）的数据
    print(var,data.shape)   #打印var（属性）的形状大小，一般用于以下range（time属性里的数据量，比如显示time（248，），则此处填248）
time=dataset.variables['time'][:].data #义数据data储存数据集里每个time的数据
real_time=[] #定义real_time为一个空列表，用来后面以对应的格式储存时间
for i in range(248):  #定义i读取248个时间段，该值由time（248，）决定
    tstamp=(time[i]-613608)*3600 
    #此处是数据读取间隔，全部需要转化为秒；比如以小时为尺度的，+/-号里面的是小时数（小时数=nc数据集最早开始的年份1/1 0:0-
    #1970年1/1 0:0，网页上有时间计算程序），后边乘号是3600转换为秒；若是以天为尺度的，+/-号里面是天数(同理)，后边则是乘以转化为秒的数值3600*24，以此类推
    date= datetime.datetime.utcfromtimestamp(tstamp) #将tstamp的时间戳转换成标准时间格式并储存在date；
    #print (date.strftime("%Y-%m-%d %H:%M:%S"))  #可打印出时间间隔
    real_time.append(date.strftime("%Y-%m-%d %H:%M:%S")) #此处将由日期格式转化为字符串格式，并储存在real_time列表中

In [ ]:
with open('Prec_197101'+'.csv','a',newline='') as targetFile: #创建一个名为（Prec_197101.csv）的文件
            # 创建写入流
        writer = csv.writer(targetFile)
            # 写入表头
        writer.writerow(('lat', 'lon', 'time',  'prec',
                             ))
            # 写入数据
        for j in range(len(lat)):  # j为纬度的间隔数，间隔数越大，纬度越大；len定义维度个数（nc格式的维度是分数量的，每隔多少一个间隔），
            for k in range(len(lon)):  # k为经度间隔数，同上
                for l in range(len(real_time)): #时间段
                    if lat[j] > 23 and lat[j] < 25:  #假设满足条件（纬度范围23-25），即开始判断下一个条件（经度范围118-119），则开始写入降水量，以j，k为间隔读取经纬度
                        if lon[k] > 118 and lon[k] < 119:   #定义经度范围18-119
                            writer.writerow((lat[j], lon[k], real_time[l], prec[l][j][k])) #写入逐一读取的经纬度，相应的时间段，降水量
        targetFile.close()#关闭文件
        print('Get'+file_name+'.csv Success!') #如果处理完成，则输出csv成功